In [28]:
import numpy as np
import yaml
import pandas as pd    
import math

from dataclasses import dataclass
from typing import List, Optional, Tuple
import itertools 
from docopt import docopt
from prettytable import PrettyTable 
from rich.prompt import Prompt
from datetime import datetime, timedelta

In [2]:
import logging
logging.basicConfig()
LOGGER = logging.getLogger(__name__)
LOGGER.setLevel(logging.INFO)


In [3]:
def calculate_1rm(reps, weight, rpe):
    """
    Use Brzycki's formula to calculate the theortical 1RM
    """
    one_rm_pct = DF_RPE[f'{reps}'].to_dict().get(rpe)
    one_rm = int(np.round(np.floor(weight / one_rm_pct), 0))
    return one_rm

def round_weights(weight, deload=False):
    """
    Round the weights to the nearest 2.5 kg (or kg if microplates available)
    """
    if deload: 
        pl_weight = MIN_PLATE_WEIGHT * math.floor(weight / MIN_PLATE_WEIGHT)
    else: 
        pl_weight = MIN_PLATE_WEIGHT * math.ceil(weight / MIN_PLATE_WEIGHT)

    return pl_weight
            

In [4]:

@dataclass
class OneRepMax:
    """A dataclass to store the one rep max for a lift"""

    lift: str
    weight: float
        
    @property
    def output_table(self) -> str:
        """Return a table of the one rep max"""
        output = [self.lift.title(), self.weight]
        return output




In [5]:
def get_one_rm_pct(reps, rpe): 
    "Get the percentage of 1RM for a given number of reps and RPE"
    one_rm_pct = DF_RPE[f'{reps}'].to_dict().get(rpe)
    return one_rm_pct

In [6]:
def calculate_training_range(one_rm, reps, rpe_schema) -> List[float]:
    """
    Calculate the training day progression for a given 1RM
    """
    week_weights = []
    
    for rpe in rpe_schema:
        pct_one_rm = get_one_rm_pct(reps, rpe)
        week_weight = pct_one_rm * one_rm
        week_weights.append(week_weight)

    training_range = [round_weights(i) for i in week_weights]
        
    return training_range

In [7]:
proposal = [40.0, 42.5, 42.5, 45.0, 45.0]
weekly_difference = np.diff(proposal).tolist()
for i, weekly_diff in enumerate(weekly_difference, 0): 
    if weekly_diff < 2.5: 
        proposal[i+1] = proposal[i+1] + 2.5
        # update weekly difference value 
        weekly_difference = np.diff(proposal).tolist()
proposal

[40.0, 42.5, 45.0, 45.0, 47.5]

In [15]:

def create_training_range(one_rm, sets, reps, rpe_schema) -> List[LiftWeight]:
    """Create a list of LiftWeight objects for the training range"""
    training_range = calculate_training_range(one_rm, reps, rpe_schema)

    weights_lifted = list(
        zip(training_range, itertools.repeat(sets), itertools.repeat(reps))
    )
    weights_lifted = [LiftWeight(*i) for i in weights_lifted]

    return weights_lifted

In [16]:


def calculate_training_range(one_rm, reps, rpe_schema) -> List[float]:
    """
    Calculate the training day progression for a given 1RM
    """
    lowest_rpe = rpe_schema[0]
    highest_rpe = rpe_schema[-1]

    w1_weight = round_weights(get_one_rm_pct(reps, lowest_rpe) * one_rm, deload=True)
    w5_weight = get_one_rm_pct(reps, highest_rpe) * one_rm

    week_weights = np.linspace(w1_weight, w5_weight, 5).tolist()

    training_range = [round_weights(i) for i in week_weights]

    # check if there is minimum spacing between each week

    return training_range

In [17]:
def create_weekly_dates(start_date): 
    """
    Create the weekly dates for the program
    """
    starting_monday = datetime.date(
        start_date + timedelta(days=7 - start_date.weekday())
    )
    
    week_mondays = []
    for week in range(1, 6):
        week_monday = starting_monday + timedelta(days=7*week)
        week_mondays.append(week_monday)

    # Create the week dates as str 
    week_dates = [i.strftime('%d/%m/%y') for i in week_mondays]                
    return week_dates

In [31]:
user_profile='default'
if __name__ == "__main__":    
    defined_rpes = yaml.load(open("source/rpe.yaml", "r"), Loader=yaml.FullLoader)
    exercises = yaml.load(open("config/exercise.json", "r"), Loader=yaml.FullLoader)
    program_layout = yaml.load(
        open("config/program-layout.yaml", "r"), Loader=yaml.FullLoader
    )
    accessory_rpe_schema = defined_rpes.get("accessory").get("backoff")

    LOGGER.info("Reading user inputs")
    user_lifts = yaml.load(
        open(f"profiles/{user_profile}/lifts.yaml", "r"), Loader=yaml.FullLoader
    )
    try:
        user_gym = yaml.load(
            open(f"profiles/{user_profile}/gym.yaml", "r"), Loader=yaml.FullLoader
        )
    except FileNotFoundError:
        user_gym = False

    # Global variables
    DF_RPE = pd.read_csv("source/rpe-calculator.csv").set_index("RPE")
    MICROPLATES = user_gym.get("microplates") if user_gym else False
    MIN_PLATE_WEIGHT = 1 if MICROPLATES else 2.5
    ACCESSORY_SETS = exercises.get("accessory-config").get("max-sets")

    max_lifts = []
    full_program = []

    for compound_lift_name, stats in user_lifts.items():
        # clear previous stats
        LOGGER.info("Creating program for %s", compound_lift_name)
        top_training_range = []
        backoff_training_range = []

        lift_one_rep_max = calculate_1rm(
            reps=stats.get("reps"), rpe=stats.get("rpe"), weight=stats.get("weight")
        )

        max_strength = OneRepMax(lift=compound_lift_name, weight=lift_one_rep_max)

        goal = stats.get("program-goal")
        program_type = goal.split("-")[0]
        rpes = defined_rpes.get(program_type)
        exercise_volume = exercises.get("program-goals").get(f"{goal}")

        if program_type == "strength":
            top_training_range = create_training_range(
                one_rm=lift_one_rep_max,
                sets=exercise_volume.get("top-sets"),
                reps=exercise_volume.get("top-reps"),
                rpe_schema=rpes.get("top"),
            )

        backoff_training_range = create_training_range(
            one_rm=lift_one_rep_max,
            sets=exercise_volume.get("backoff-sets"),
            reps=exercise_volume.get("backoff-reps"),
            rpe_schema=rpes.get("backoff"),
        )

        compound_lift = Exercise(
            backoff_sets=backoff_training_range,
            name=compound_lift_name,
            predicted_one_rm=lift_one_rep_max,
            top_sets=top_training_range,
        )

        full_program.append(compound_lift)

        accessories = exercises.get("accessory-lifts").get(f"{compound_lift_name}")
        for accessory, stats in accessories.items():
            accessory_one_rm = round_weights(
                lift_one_rep_max * stats.get("max-onerm-pct")
            )
            min_reps = stats.get("min-reps")

            accessory_training_range = create_training_range(
                one_rm=accessory_one_rm,
                sets=ACCESSORY_SETS,
                reps=min_reps,
                rpe_schema=accessory_rpe_schema,
            )

            accessory_lift = Exercise(
                backoff_sets=accessory_training_range,
                name=accessory,
                predicted_one_rm=accessory_one_rm,
                related_compound=compound_lift,
            )

            full_program.append(accessory_lift)

        max_lifts.append(max_strength)

     # create the program
    LOGGER.info("Creating list of available exercises to output")
    all_exercises = {}
    for exercise in full_program:
        all_exercises[exercise.name] = exercise

INFO:__main__:Reading user inputs
INFO:__main__:Creating program for squat
INFO:__main__:Creating program for bench
INFO:__main__:Creating program for deadlift
INFO:__main__:Creating list of available exercises to output


In [32]:
user_stats = PrettyTable()
user_stats.field_names = ['Lift', 'Training Max']
for lift in max_lifts:
    max_tm_row = lift.output_table
    user_stats.add_row(max_tm_row)

user_stats

Lift,Training Max
Squat,95
Bench,58
Deadlift,132


In [33]:
max_exercises = len(max(list(program_layout.values()), key=len))

In [34]:
def column_pad(*columns):
  max_len = max([len(c) for c in columns])
  for c in columns:
      c.extend(['']*(max_len-len(c)))

In [41]:
day_cols = []
for _, exercises in program_layout.items(): 
    day_col = []
    for exercise_no, exercise_name in enumerate(exercises, 1): 
        is_first_exercise = 1 if exercise_no == 1 else 0
        planned_exercise = all_exercises.get(exercise_name)
        if planned_exercise: 
            exercise_col = planned_exercise.output_table
        else:
            generic_exericse=Exercise(
                backoff_sets= [LiftWeight(0, ACCESSORY_SETS, 12)] * 5,
                name=exercise_name,
            )
            exercise_col = generic_exericse.output_table
        if is_first_exercise:
            print(exercise_col)
            # if len(exericse_col) == 7: 

            
        day_col.append(exercise_col)
    # print(day_col)
    flatten_day_col = [item for sublist in day_col for item in sublist]

    day_cols.append(flatten_day_col)

column_pad(*day_cols)

program = PrettyTable()
# week_dates=create_weekly_dates(start_date)
# week_dates.insert(0, "Week Starting")
# week_dates.append("")
# five_week_dates= week_dates * 5
# program.add_column("", five_week_dates)

# for day, col in enumerate(day_cols, 1): 
#     program.add_column(f"Day {day}", col)


['Squat', ('1 x 1', '82.5'), ('3 x 3', '72.5'), ('1 x 1', '87.5'), ('3 x 3', '75.0'), ('1 x 1', '90.0'), ('3 x 3', '77.5'), ('1 x 1', '92.5'), ('3 x 3', '80.0'), ('1 x 1', '95.0'), ('3 x 3', '82.5'), '']
['Bench', ('3 x 8', '37.5'), ('3 x 8', '40.0'), ('3 x 8', '42.5'), ('3 x 8', '45.0'), ('3 x 8', '45.0'), '']
['Deadlift', ('3 x 6', '97.5'), ('3 x 6', '100.0'), ('3 x 6', '102.5'), ('3 x 6', '105.0'), ('3 x 6', '107.5'), '']


In [64]:
week_dates = ['2022-12-07', '2022-12-08', '2022-12-09', '2022-12-10', '2022-12-11', '2022-12-12', '2022-12-13'] 

In [65]:
week_dates = list(itertools.chain.from_iterable(zip(*[week_dates]*2)))

In [42]:
start_date = datetime.now()

In [43]:
def create_empty_column_with_header(
    column_to_create: str, 
    column_length: int
) -> list: 
    empty_entries = [""] * 6
    # first exercise will have double number of entries because of top and backoff sets
    first_exercise = [f"{column_to_create}"] + empty_entries * 2

    # every subsequent exercise does not have top sets
    single_col = [f"{column_to_create}"] + empty_entries
    single_cols = first_exercise + single_col * (column_length-1)

    return single_cols

In [48]:
column_to_create = "notes"
column_length=5
empty_entries = [""] * 6
first_exercise = [f"{column_to_create}"] + empty_entries * 2
single_col = [f"{column_to_create}"] + empty_entries
single_cols = first_exercise + single_col * (column_length-1)

In [49]:
single_cols

['notes',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'notes',
 '',
 '',
 '',
 '',
 '',
 '',
 'notes',
 '',
 '',
 '',
 '',
 '',
 '',
 'notes',
 '',
 '',
 '',
 '',
 '',
 '',
 'notes',
 '',
 '',
 '',
 '',
 '',
 '']

In [ ]:

week_dates=create_weekly_dates(start_date)
week_dates.insert(0, "Week Starting")
week_dates.append("")
five_week_dates= week_dates * 5
program.add_column("", five_week_dates)

In [132]:
deadlift_col = ['Deadlift', ('3 x 6', '97.5'), ('3 x 6', '100.0'), ('3 x 6', '102.5'), ('3 x 6', '105.0'), ('3 x 6', '107.5'), '']
pretty_exercise_col = []
for index, item in enumerate(deadlift_col, 0): 
    if isinstance(item, tuple): 
        pretty_exercise_col.append(item)


In [ ]:
list(itertools.zip_longest(itertools.repeat(""), pretty_exercise_col))

In [37]:

@dataclass
class LiftWeight:
    """A dataclass to store the weight for a lift"""
    weight: float
    sets: int
    reps: int

    @property 
    def pretty_name(self) -> tuple:
        """
        Return a tuple for the lift
        """
        return (f"{self.sets} x {self.reps}", f"{self.weight}")

@dataclass
class Exercise: 
    """A generic class for an exercise"""
    backoff_sets: List[LiftWeight]
    name: str
    predicted_one_rm: float = None
    related_compound: str = None
    top_sets: Optional[List[LiftWeight]] = None
    
    @property 
    def pretty_name(self) -> str:
        """Clean the names / related compound names"""
        exercise_name = self.name.replace('-', ' ').title()
        return exercise_name

    @property 
    def weekly_sets(self) -> List[LiftWeight]:
        """
        Return the weekly training set for the exercise
        """
        if self.top_sets:
            weekly_sets = list(itertools.zip_longest(self.top_sets, self.backoff_sets))
            return weekly_sets
        else: 
            return self.backoff_sets
    
    @property 
    def make_weekly_sets(self) -> List[Tuple]: 
        weekly = []
        if self.top_sets:
            for _, (top, backoff) in enumerate(self.weekly_sets, 1):
                top_sets = top.pretty_name 
                backoff_sets = backoff.pretty_name
                weekly.extend([top_sets, backoff_sets])
        else:
            for _, weight in enumerate(self.weekly_sets, 1):
                backoff_sets = weight.pretty_name
                weekly.extend([backoff_sets])
        return weekly

    @property
    def output_table(self) -> List:
        "Creates table output for prettytable package"
        outputs = [f"{self.pretty_name}"]
        outputs.extend(self.make_weekly_sets)
        # add empty row for formatting
        outputs.append("")
        return outputs

In [56]:
# create weekly dates

program

,Day 1,Day 2,Day 3
Week Starting,Squat,Bench,Deadlift
19/12/22,3 x 8 | 62.5 kg,3 x 8 | 37.5 kg,3 x 6 | 97.5 kg
26/12/22,3 x 8 | 65.0 kg,3 x 8 | 40.0 kg,3 x 6 | 100.0 kg
02/01/23,3 x 8 | 67.5 kg,3 x 8 | 42.5 kg,3 x 6 | 102.5 kg
09/01/23,3 x 8 | 70.0 kg,3 x 8 | 45.0 kg,3 x 6 | 105.0 kg
16/01/23,3 x 8 | 72.5 kg,3 x 8 | 45.0 kg,3 x 6 | 107.5 kg
,,,
Week Starting,Paused Bench,Paused Deadlift,Safety Squat Bar
19/12/22,3 x 3 | 37.5 kg,3 x 3 | 80.0 kg,3 x 3 | 65.0 kg
26/12/22,3 x 3 | 40.0 kg,3 x 3 | 82.5 kg,3 x 3 | 67.5 kg


In [47]:
user_stats

Lift,Training Max
Squat,95
Bench,58
Deadlift,132
